In [1]:
#create a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").\
                                     appName("spark_on_docker").\
                                     getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/28 14:46:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
flightData2015 = spark.read.csv('work/TheDefinitiveGuide/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv', header=True)
#print(flightData2015.show(5))

DataFrame을 테이블이나 뷰로 만들기 위한 createOrReplaceTempView()

In [4]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [5]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

In [6]:
print(sqlWay.show(5))

+-----------------+--------+
|DEST_COUNTRY_NAME|count(1)|
+-----------------+--------+
|         Anguilla|       1|
|           Russia|       1|
|         Paraguay|       1|
|          Senegal|       1|
|           Sweden|       1|
+-----------------+--------+
only showing top 5 rows

None


In [7]:
dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

In [8]:
print(dataFrameWay.show(5))

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
|         Anguilla|    1|
|           Russia|    1|
|         Paraguay|    1|
|          Senegal|    1|
|           Sweden|    1|
+-----------------+-----+
only showing top 5 rows

None


In [9]:
sqlWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#16, 200), ENSURE_REQUIREMENTS, [id=#123]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#16] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/TheDefinitiveGuide/Spark-The-Definitive-Guide/d..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [10]:
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#16, 200), ENSURE_REQUIREMENTS, [id=#136]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#16] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/TheDefinitiveGuide/Spark-The-Definitive-Guide/d..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [11]:
from pyspark.sql.functions import max

flightData2015.select(max("count")).take(1)


[Row(max(count)='986')]

In [12]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

In [13]:
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|         411352.0|
|           Canada|           8399.0|
|           Mexico|           7140.0|
|   United Kingdom|           2025.0|
|            Japan|           1548.0|
+-----------------+-----------------+



In [14]:
from pyspark.sql.functions import desc

In [15]:
flightData2015.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



In [16]:
from pyspark.sql.types import IntegerType
flightData2015 = flightData2015.withColumn("count", flightData2015["count"].cast(IntegerType()))
flightData2015.groupBy("DEST_COUNTRY_NAME").max("count") 

DataFrame[DEST_COUNTRY_NAME: string, max(count): int]

In [17]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [18]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#127L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#16,destination_total#127L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[sum(count#84)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#16, 200), ENSURE_REQUIREMENTS, [id=#301]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#16], functions=[partial_sum(count#84)])
            +- Project [DEST_COUNTRY_NAME#16, cast(count#18 as int) AS count#84]
               +- FileScan csv [DEST_COUNTRY_NAME#16,count#18] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/TheDefinitiveGuide/Spark-The-Definitive-Guide/d..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:string>




In [19]:
flightData2015.count()

256

In [20]:
spark.stop()